# Rag using Document ai spliter

due to problems using unstructured package to split document, I to stay in google echo-system I'll use Document AI to split a documents, thuis this notebook is for that purpose.<br>
I followed this [playlist](https://www.youtube.com/watch?v=9izcbNYmP8M&list=PLIivdWyY5sqIR88BxIK-3w14Vm-jTH1id&index=2) to get started with document AI  

In [1]:
PROJECT_ID = "x-project-00"
LOCATION = "eu"
DOCUMENT_OCR_DISPLAY_NAME = "my-layout-processor"
PROCESSOR_ID = "92d70cd187ed276"
FILE_PATH = "X3000_TurboFixer.pdf"
BUCKET_NAME = "ai_test_bckt"
MIME_TYPE = "application/pdf"

# Synchronous call 
eused for small files, the result are send to the caller after processing

In [ ]:
from google.cloud import documentai_v1beta3 as documentai
from google.cloud import storage
from google.api_core.client_options import ClientOptions

# Instantiates a client
storage_client = storage.Client()
opts = ClientOptions(api_endpoint=f"{LOCATION}-documentai.googleapis.com")

client = documentai.DocumentProcessorServiceClient(client_options=opts)
# The full resource name of the processor, e.g.:
# projects/project-id/locations/location/processor/processor-id
# You must create new processors in the Cloud Console first
name = f"projects/{PROJECT_ID}/locations/{LOCATION}/processors/{PROCESSOR_ID}"
name2 = "projects/815082548201/locations/eu/processors/92d70cd187ed276"
bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob(FILE_PATH)
with blob.open("rb") as image:
    image_content = image.read()
# Read the file into memory
document = {"content": image_content, "mime_type": "application/pdf"}
# Configure the process request
request = {"name": name, "document": document}
# Use the Document AI client to process the sample form
result = client.process_document(request=request)
result

In [ ]:
result.document

In [ ]:
import collections

pdf_categories2 = [el.category for el in elements]
print(collections.Counter(pdf_categories2))

In [ ]:
result.document.pages[0].key

In [ ]:
result.document.entities

In [ ]:
a = [(r.page_number, r.layout) for r in result.document.pages]
a

In [ ]:
from typing import Optional

from google.api_core.client_options import ClientOptions
from google.cloud import documentai  # type: ignore

# TODO(developer): Uncomment these variables before running the sample.
# project_id = "YOUR_PROJECT_ID"
# location = "YOUR_PROCESSOR_LOCATION" # Format is "us" or "eu"
# processor_id = "YOUR_PROCESSOR_ID" # Create processor before running sample
# file_path = "/path/to/local/pdf"
# mime_type = "application/pdf" # Refer to https://cloud.google.com/document-ai/docs/file-types for supported file types
# field_mask = "text,entities,pages.pageNumber"  # Optional. The fields to return in the Document object.
# processor_version_id = "YOUR_PROCESSOR_VERSION_ID" # Optional. Processor version to use


def process_document_sample(
    project_id: str,
    location: str,
    processor_id: str,
    file_path: str,
    mime_type: str,
    field_mask: Optional[str] = None,
    processor_version_id: Optional[str] = None,
) -> None:
    # You must set the `api_endpoint` if you use a location other than "us".
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    if processor_version_id:
        # The full resource name of the processor version, e.g.:
        # `projects/{project_id}/locations/{location}/processors/{processor_id}/processorVersions/{processor_version_id}`
        name = client.processor_version_path(
            project_id, location, processor_id, processor_version_id
        )
    else:
        # The full resource name of the processor, e.g.:
        # `projects/{project_id}/locations/{location}/processors/{processor_id}`
        name = client.processor_path(project_id, location, processor_id)

    # Read the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Load binary data
    raw_document = documentai.RawDocument(content=image_content, mime_type=mime_type)

    # For more information: https://cloud.google.com/document-ai/docs/reference/rest/v1/ProcessOptions
    # Optional: Additional configurations for processing.
    process_options = documentai.ProcessOptions(
        # Process only specific pages
        individual_page_selector=documentai.ProcessOptions.IndividualPageSelector(
            pages=[1]
        )
    )

    # Configure the process request
    request = documentai.ProcessRequest(
        name=name,
        raw_document=raw_document,
        field_mask=field_mask,
        process_options=process_options,
    )

    result = client.process_document(request=request)

    # For a full list of `Document` object attributes, reference this page:
    # https://cloud.google.com/document-ai/docs/reference/rest/v1/Document
    document = result.document

    # Read the text recognition output from the processor
    print("The document contains the following text:")
    print(document.text)

process_document_sample(PROJECT_ID,LOCATION, PROCESSOR_ID,FILE_PATH,"application/pdf")

# Asynchronous 
used for bigger/multiple files, the input and output are stored in gcs

In [4]:
from google.cloud import documentai_v1beta3 as documentai
from google.cloud import storage
from google.api_core.client_options import ClientOptions

client_options = {"api_endpoint": "{}-documentai.googleapis.com".format(LOCATION)}
client = documentai.DocumentProcessorServiceClient(client_options=client_options)
storage_client = storage.Client()

In [7]:
blobs = storage_client.list_blobs(BUCKET_NAME)
input_configs = []
print("Input Files:")
for blob in blobs:
    if ".pdf" in blob.name and FILE_PATH in blob.name:
        source = "gs://{bucket}/{name}".format(bucket = BUCKET_NAME, name = blob.name)
        print(source)
        input_config = documentai.types.document_processor_service.BatchProcessRequest.BatchInputConfig(
            gcs_source=source, mime_type="application/pdf"
        )
        input_configs.append(input_config)

Input Files:
gs://ai_test_bckt/X3000_TurboFixer.pdf


In [9]:
destination_uri = f"gs://{BUCKET_NAME}/out"
output_config = documentai.types.document_processor_service.BatchProcessRequest.BatchOutputConfig(
    gcs_destination=destination_uri
)

In [11]:
name = f"projects/{PROJECT_ID}/locations/{LOCATION}/processors/{PROCESSOR_ID}"

request = documentai.types.document_processor_service.BatchProcessRequest(
    name=name,
    input_configs=input_configs,
    output_config=output_config,
)

In [12]:
operation = client.batch_process_documents(request)
# Wait for the operation to finish
operation.result(timeout=800)
print (f"Batch process  completed. the output will be in {BUCKET_NAME}/out")

Batch process  completed.
